In [1]:
import numpy as np
import optuna
import joblib
import ast
from sklearn.metrics import f1_score
import os

In [2]:

import ast
from sklearn.metrics import f1_score, classification_report, precision_score, ndcg_score
def criterion_evaluation(path, max_round, criterion, thresholds):
    f1= 0
    with open(f"{path}/decisions.txt", "r") as f:
        true_labels = []
        classifications = []
        ranking_score=[]
        for line in f:
            splt = line[:-1].split(" ")
            splt = [ast.literal_eval(t) for t in splt]
            user, label = splt[0]
            true_labels.append(label)
            classification = 0
            confidence_score=0
            count = 0
            for i, (pred, conf) in enumerate(splt[1+9:]):
                if i>=max_round:
                        break
                if criterion == "confidence":#  and i%10==0:
                    if pred==1 and conf >= thresholds['confidence_threshold'] :
                        classification= 1
                        confidence_score+=conf
                if criterion == "ratio" and conf >=thresholds['confidence_threshold']:# and i%10==0:
                    count = count + pred
                    if count/(10 + i) >= thresholds['ratio_threshold']:
                        classification=1
                        confidence_score+=conf

                if criterion == "confidence+" and conf >=thresholds['confidence_threshold']:# and i%10==0:
                    count = count + pred
                    if count >= thresholds['amount_threshold']:
                        classification=1
                        confidence_score+=conf

                if criterion == "consecutive":# and i%10==0: ##added stride
                    if pred==1 and conf >=thresholds['confidence_threshold']:
                        count = count + 1

                    else:
                        count = 0
                    if count >= thresholds['consecutive_threshold']:
                        classification = 1
                        confidence_score+=conf

            ranking_score.append((confidence_score, label))
            classifications.append(classification)
    f1=f1_score(true_labels,classifications)
    cls_rep = classification_report(true_labels,classifications, digits=3, output_dict=True)['1']
    ranking_score = sorted(ranking_score, key=lambda tup:tup[0], reverse=True)
    true_ranking_labels = [i[1] for i in ranking_score]
    ranking_scores = [i[0] for i in ranking_score]
    #print(true_ranking_labels[:10], [1 for i in ranking_scores[:10] if i>0])
    p_at_10 = precision_score(true_ranking_labels[:10], [1 for i in ranking_scores[:10] if i>0])

    
    ndcg_at_10 = ndcg_score([true_ranking_labels[:10]], [ranking_scores[:10]])
    
    
    ndcg_at_100 = ndcg_score([true_ranking_labels[:100]], [ranking_scores[:100]])

    #print(ranking_score)
    stats = (cls_rep,f1,p_at_10,ndcg_at_10,ndcg_at_100)
    return stats

In [3]:
def ensemble_criterion_evaluation(path1,path2,path3, max_round, criterion, thresholds):
    f1= 0
    with open(f"{path1}/decisions.txt", "r") as file1:
        with open(f"{path2}/decisions.txt", "r") as file2:
            with open(f"{path3}/decisions.txt", "r") as file3:

                true_labels = {}
                classifications1 = {}
                confidence_scores1={}
                classifications2 = {}
                confidence_scores2={}
                classifications3 = {}
                confidence_scores3={}


                for line in file1:
                    splt = line[:-1].split(" ")
                    splt = [ast.literal_eval(t) for t in splt]
                    user, label = splt[0]
                    true_labels[user]=label
                    classification = 0
                    confidence_score=0
                    count = 0
                    for i, (pred, conf) in enumerate(splt[10:]):
                        if i>=max_round:
                                break
                        if criterion == "confidence":#  and i%10==0:
                            if pred==1 and conf >= thresholds[0]['confidence_threshold'] :
                                classification= 1
                                confidence_score+=conf
                        if criterion == "ratio" and conf >=thresholds[0]['confidence_threshold']:# and i%10==0:
                            count = count + pred
                            if count/(10 + i) >= thresholds[0]['ratio_threshold']:
                                classification=1
                                confidence_score+=conf

                        if criterion == "confidence+" and conf >=thresholds[0]['confidence_threshold']:# and i%10==0:
                            count = count + pred
                            if count >= thresholds[0]['amount_threshold']:
                                classification=1
                                confidence_score+=conf

                        if criterion == "consecutive":# and i%10==0: ##added stride
                            if pred==1 and conf >=thresholds[0]['confidence_threshold']:
                                count = count + 1

                            else:
                                count = 0
                            if count >= thresholds[0]['consecutive_threshold']:
                                classification = 1
                                confidence_score+=conf

                        
                    classifications1[user]=classification
                    confidence_scores1[user]=confidence_score

                    for line in file2:
                        splt = line[:-1].split(" ")
                        splt = [ast.literal_eval(t) for t in splt]
                        user, label = splt[0]
                        true_labels[user]=label
                        classification = 0
                        confidence_score=0

                        count = 0
                        for i, (pred, conf) in enumerate(splt[10:]):
                            if i>=max_round:
                                    break
                            if criterion == "confidence":#  and i%10==0:
                                if pred==1 and conf >= thresholds[1]['confidence_threshold'] :
                                    classification= 1
                                    confidence_score+=conf
                            if criterion == "ratio" and conf >=thresholds[1]['confidence_threshold']:# and i%10==0:
                                count = count + pred
                                if count/(10 + i) >= thresholds[1]['ratio_threshold']:
                                    classification=1
                                    confidence_score+=conf

                            if criterion == "confidence+" and conf >=thresholds[1]['confidence_threshold']:# and i%10==0:
                                count = count + pred
                                if count >= thresholds[1]['amount_threshold']:
                                    classification=1
                                    confidence_score+=conf

                            if criterion == "consecutive":# and i%10==0: ##added stride
                                if pred==1 and conf >=thresholds[1]['confidence_threshold']:
                                    count = count + 1

                                else:
                                    count = 0
                                if count >= thresholds[1]['consecutive_threshold']:
                                    classification = 1
                                    confidence_score+=conf
   
                        classifications2[user]=classification
                        confidence_scores2[user]=confidence_score

                    for line in file3:
                        splt = line[:-1].split(" ")
                        splt = [ast.literal_eval(t) for t in splt]
                        user, label = splt[0]
                        true_labels[user]=label
                        classification = 0
                        confidence_score=0
                        count = 0
                        for i, (pred, conf) in enumerate(splt[10:]):
                            if i>=max_round:
                                    break
                            if criterion == "confidence":#  and i%10==0:
                                if pred==1 and conf >= thresholds[2]['confidence_threshold'] :
                                    classification= 1
                                    confidence_score+=conf
                            if criterion == "ratio" and conf >=thresholds[2]['confidence_threshold']:# and i%10==0:
                                count = count + pred
                                if count/(10 + i) >= thresholds[2]['ratio_threshold']:
                                    classification=1
                                    confidence_score+=conf

                            if criterion == "confidence+" and conf >=thresholds[2]['confidence_threshold']:# and i%10==0:
                                count = count + pred
                                if count >= thresholds[2]['amount_threshold']:
                                    classification=1
                                    confidence_score+=conf

                            if criterion == "consecutive":# and i%10==0: ##added stride
                                if pred==1 and conf >=thresholds[2]['confidence_threshold']:
                                    count = count + 1

                                else:
                                    count = 0
                                if count >= thresholds[2]['consecutive_threshold']:
                                    classification = 1
                                    confidence_score+=conf
    
                        classifications3[user]=classification
                        confidence_scores3[user]=confidence_score
    classifications = {}
    ranking_scores=[] 
    for user in true_labels:
        
        if classifications1[user] + classifications2[user] + classifications3[user]>=2:
            classifications[user]=1
        else:
            classifications[user]=0
        ranking_scores.append((confidence_scores1[user] + confidence_scores2[user] + confidence_scores3[user],classifications[user],true_labels[user]))
    f1=f1_score(list(true_labels.values()),list(classifications.values()))
    cls_rep = classification_report(list(true_labels.values()),list(classifications.values()), digits=3, output_dict=True)['1']

    ranking_scores = sorted(ranking_scores, key=lambda tup:tup[0], reverse=True)
    true_ranking_labels = [i[2] for i in ranking_scores]
    ranking_labels = [i[1] for i in ranking_scores]
    ranking_scores = [i[0] for i in ranking_scores]
    #print(true_ranking_labels[:10], [1 for i in ranking_scores[:10] if i>0])
    p_at_10 = precision_score(true_ranking_labels[:10], [1 for i,_ in enumerate(ranking_scores[:10]) if ranking_labels[i]>0])

    
    ndcg_at_10 = ndcg_score([true_ranking_labels[:10]], [ranking_scores[:10]])
    
    
    ndcg_at_100 = ndcg_score([true_ranking_labels[:100]], [ranking_scores[:100]])

    #print(ranking_score)
    stats = (cls_rep,f1,p_at_10,ndcg_at_10,ndcg_at_100)
    return stats

In [4]:
max_round=100


In [5]:
##loading thresholds
import joblib
def get_thresholds(model):
    study_conf=joblib.load(f"../Thresholds2/{model}_confidence_100.pkl")
    study_confplus=joblib.load(f"../Thresholds2/{model}_confidence+_100.pkl")
    study_ratio=joblib.load(f"../Thresholds2/{model}_ratio_100.pkl")
    study_consec=joblib.load(f"../Thresholds2/{model}_consecutive_100.pkl")

    conf_thresh = study_conf.best_trial.params
    confplus_thresh = study_confplus.best_trial.params
    ratio_thresh = study_ratio.best_trial.params
    consec_thresh = study_consec.best_trial.params

    return {'confidence':conf_thresh,'confidence+':confplus_thresh,'ratio':ratio_thresh,'consecutive':consec_thresh}


In [6]:
from tabulate import tabulate
with open("report2.txt","w",encoding="utf-8") as f:
    decision_table=[['model','criterion','precision','recall','f1']]
    ranking_table=[['model','criterion','precision@10','ndcg@10','ndcg@100']]
    for model in ['BoW','WE','LM','NN','Ensemble']:
        #print(model)
        #f.write(f"###{model}###\n")
        if model == 'Ensemble':
            thresholds_bow = get_thresholds("BoW")
            thresholds_we = get_thresholds("WE")
            thresholds_lm = get_thresholds("LM")
            
            thresholds = {0:thresholds_bow["confidence"],1:thresholds_we["confidence"],2:thresholds_lm["confidence"]}
            rep,f1,p_at_10,ndcg_at_10,ndcg_at_100=ensemble_criterion_evaluation("BoW","WE","LM", 100, "confidence",thresholds)
            decision_table.append([model,"confidence",rep["precision"],rep["recall"], rep["f1-score"]])
            ranking_table.append([model,"confidence", p_at_10,ndcg_at_10,ndcg_at_100])
            
            thresholds = {0:thresholds_bow["confidence+"],1:thresholds_we["confidence+"],2:thresholds_lm["confidence+"]}
            rep,f1,p_at_10,ndcg_at_10,ndcg_at_100=ensemble_criterion_evaluation("BoW","WE","LM", 100, "confidence+",thresholds)
            decision_table.append([model,"confidence+",rep["precision"],rep["recall"], rep["f1-score"]])
            ranking_table.append([model,"confidence+", p_at_10,ndcg_at_10,ndcg_at_100])
            
            thresholds = {0:thresholds_bow["ratio"],1:thresholds_we["ratio"],2:thresholds_lm["ratio"]}
            rep,f1,p_at_10,ndcg_at_10,ndcg_at_100=ensemble_criterion_evaluation("BoW","WE","LM", 100, "ratio",thresholds)
            decision_table.append([model,"ratio",rep["precision"],rep["recall"], rep["f1-score"]])
            ranking_table.append([model,"ratio", p_at_10,ndcg_at_10,ndcg_at_100])
            
            thresholds = {0:thresholds_bow["consecutive"],1:thresholds_we["consecutive"],2:thresholds_lm["consecutive"]}
            rep,f1,p_at_10,ndcg_at_10,ndcg_at_100=ensemble_criterion_evaluation("BoW","WE","LM", 100, "consecutive",thresholds)
            decision_table.append([model,"consecutive",rep["precision"],rep["recall"], rep["f1-score"]])
            ranking_table.append([model,"consecutive", p_at_10,ndcg_at_10,ndcg_at_100])

        else:
            thresholds = get_thresholds(model)
            rep,f1,p_at_10,ndcg_at_10,ndcg_at_100 = criterion_evaluation(model,max_round,"confidence", thresholds["confidence"])
            decision_table.append([model,"confidence",rep["precision"],rep["recall"], rep["f1-score"]])
            ranking_table.append([model,"confidence", p_at_10,ndcg_at_10,ndcg_at_100])
            #f.write(f"confidence\nthreshold {thresholds['confidence']}:\n{rep['1']}\nf1:{f1}\n\n")
            rep,f1,p_at_10,ndcg_at_10,ndcg_at_100= criterion_evaluation(model,max_round,"confidence+", thresholds["confidence+"])
            decision_table.append([model,"confidence+",rep["precision"],rep["recall"], rep["f1-score"]])
            ranking_table.append([model,"confidence+", p_at_10,ndcg_at_10,ndcg_at_100])

            #f.write(f"confidence+\nthreshold {thresholds['confidence+']}:\n{rep['1']}\nf1:{f1}\n\n")
            rep,f1,p_at_10,ndcg_at_10,ndcg_at_100= criterion_evaluation(model,max_round,"ratio", thresholds["ratio"])
            #f.write(f"ratio\nthreshold {thresholds['ratio']}:\n{rep['1']}\nf1:{f1}\n\n")
            decision_table.append([model,"ratio",rep["precision"],rep["recall"], rep["f1-score"]])
            ranking_table.append([model,"ratio", p_at_10,ndcg_at_10,ndcg_at_100])

            rep,f1,p_at_10,ndcg_at_10,ndcg_at_100= criterion_evaluation(model,max_round,"consecutive", thresholds["consecutive"])
            decision_table.append([model,"consecutive",rep["precision"],rep["recall"], rep["f1-score"]])
            ranking_table.append([model,"consecutive", p_at_10,ndcg_at_10,ndcg_at_100])

        #f.write(f"consecutive\nthreshold {thresholds['consecutive']}:\n{rep['1']}\nf1:{f1}\n\n")
    print(tabulate(decision_table,headers='firstrow', tablefmt='fancy_grid')) 
    print(tabulate(ranking_table,headers='firstrow', tablefmt='fancy_grid'))
    f.write("decision based results\n")
    f.write(tabulate(decision_table,headers='firstrow', tablefmt='fancy_grid'))
    f.write("\nranking based results\n")
    f.write(tabulate(ranking_table,headers='firstrow', tablefmt='fancy_grid'))


╒══════════╤═════════════╤═════════════╤══════════╤══════════╕
│ model    │ criterion   │   precision │   recall │       f1 │
╞══════════╪═════════════╪═════════════╪══════════╪══════════╡
│ BoW      │ confidence  │    0.9375   │ 0.740741 │ 0.827586 │
├──────────┼─────────────┼─────────────┼──────────┼──────────┤
│ BoW      │ confidence+ │    0.846154 │ 0.814815 │ 0.830189 │
├──────────┼─────────────┼─────────────┼──────────┼──────────┤
│ BoW      │ ratio       │    0.857143 │ 0.814815 │ 0.835443 │
├──────────┼─────────────┼─────────────┼──────────┼──────────┤
│ BoW      │ consecutive │    0.863014 │ 0.777778 │ 0.818182 │
├──────────┼─────────────┼─────────────┼──────────┼──────────┤
│ WE       │ confidence  │    0.412587 │ 0.728395 │ 0.526786 │
├──────────┼─────────────┼─────────────┼──────────┼──────────┤
│ WE       │ confidence+ │    0.402685 │ 0.740741 │ 0.521739 │
├──────────┼─────────────┼─────────────┼──────────┼──────────┤
│ WE       │ ratio       │    0.477612 │ 0.790123 │ 0.5